# The Silent Bug I Find in Most Triggers

* Source: https://www.brentozar.com/archive/2019/05/the-silent-bug-i-find-in-most-triggers
* Last updated: 2019/05/05
* License: [CC BY-SA 3.0: Creative Commons Attribution-ShareAlike 3.0 Unported](https://creativecommons.org/licenses/by-sa/3.0/)
* Data contents: [Stack Overflow data dump](https://archive.org/details/stackexchange)
* Requires: any supported version of SQL Server or Azure SQL DB, any Stack Overflow database

Most triggers I run across have a really, really dangerous bug. Let's say we want to add a trigger on the Stack Overflow database's Users table. Whenever someone's Reputation is over 1,000 points, we're going to set their AboutMe to declare that they're famous:

In [17]:
CREATE OR ALTER TRIGGER trUserIsFamous ON dbo.Users
AFTER UPDATE
AS
BEGIN
DECLARE @Id INT, @Reputation INT;
SELECT @Id = Id, @Reputation = Reputation
  FROM INSERTED;

IF @Reputation > 1000
   UPDATE dbo.Users
   SET AboutMe = 'FAMOUS FOR A HIGH REPUTATION!'
   FROM dbo.Users u
   WHERE Id = @Id;
END
GO

Commands completed successfully.

Total execution time: 00:00:00.0188990

That trigger assumes only one row will be in the DELETED table, which would be true - as long as we only update one row at a time. But what if a query updates multiple rows in a single transaction?

Let's say we want to award a free reputation point to everyone in Brazil. Here are some of the people we're going to affect:

In [18]:
SELECT TOP 20 *
  FROM dbo.Users
  WHERE Location = 'Brazil';

(20 rows affected)

Total execution time: 00:00:00.6697190

Id,AboutMe,Age,CreationDate,DisplayName,DownVotes,EmailHash,LastAccessDate,Location,Reputation,UpVotes,Views,WebsiteUrl,AccountId
626,"I'm the creator of Sinática and the renowned Sinática Monitor for Firebird. Besides computers, I enjoy trying new espressos.",NULL,2008-08-07 11:52:05.957,Douglas Tosi,3,NULL,2018-01-12 16:44:17.287,Brazil,1812,154,287,http://www.sinatica.com,484
727,Just a developer wannabe.,NULL,2008-08-08 12:16:17.820,Fabio Gomes,45,NULL,2018-09-01 18:52:56.917,Brazil,3666,596,800,http://yacoding.blogspot.com,561
802,<p>Just someone trying to learn something...</p>,NULL,2008-08-09 02:50:52.577,Flávio Amieiro,7,NULL,2018-06-04 09:13:58.670,Brazil,14495,258,678,http://flavioamieiro.com,612
1100,,NULL,2008-08-12 11:58:29.663,Daniel Silveira,14,NULL,2013-09-14 02:52:14.097,Brazil,15929,295,1461,http://enerjy.sys-con.com/author/6412,839
1213,,NULL,2008-08-13 13:18:30.890,André Casteliano,1,NULL,2018-08-29 17:37:35.787,Brazil,431,131,76,NULL,922
2015,"<p>Product Manager and Developer at a financial technology provider.</p> <p>Living and working in Rio de Janeiro, Brazil</p>",NULL,2008-08-19 19:42:43.420,Gustavo,12,NULL,2018-08-21 13:45:51.207,Brazil,601,576,235,,1509
2267,,NULL,2008-08-21 12:23:14.943,João Vieira,0,NULL,2018-09-01 13:26:59.693,Brazil,931,197,102,http://www.sia.net.br,1679
2274,NULL,NULL,2008-08-21 12:44:42.223,Fernando Barrocal,71,NULL,2018-08-31 14:46:27.387,Brazil,6284,107,529,http://fernando.barrocal.com/,1685
2333,,NULL,2008-08-21 16:56:11.823,Felipe,0,NULL,2008-09-28 07:38:25.930,Brazil,56,5,14,http://felipc.com,1725
3031,Developer!,NULL,2008-08-26 12:46:22.053,zaca,0,NULL,2012-10-09 13:20:45.547,Brazil,524,115,53,http://thebrainfarm.net,2204


Many of them have reputations over 1,000 points, so those folks are all going to be set to famous, right? Let's see what happens when we run the update:

In [19]:
UPDATE dbo.Users
  SET Reputation = Reputation + 1
  WHERE Location = 'Brazil';

(4250 rows affected)

Total execution time: 00:00:00.4284660

And then check to see their new famous AboutMe:

In [20]:
SELECT TOP 20 *
  FROM dbo.Users
  WHERE Location = 'Brazil';

(20 rows affected)

Total execution time: 00:00:00.0052030

Id,AboutMe,Age,CreationDate,DisplayName,DownVotes,EmailHash,LastAccessDate,Location,Reputation,UpVotes,Views,WebsiteUrl,AccountId
626,"I'm the creator of Sinática and the renowned Sinática Monitor for Firebird. Besides computers, I enjoy trying new espressos.",NULL,2008-08-07 11:52:05.957,Douglas Tosi,3,NULL,2018-01-12 16:44:17.287,Brazil,1813,154,287,http://www.sinatica.com,484
727,Just a developer wannabe.,NULL,2008-08-08 12:16:17.820,Fabio Gomes,45,NULL,2018-09-01 18:52:56.917,Brazil,3667,596,800,http://yacoding.blogspot.com,561
802,<p>Just someone trying to learn something...</p>,NULL,2008-08-09 02:50:52.577,Flávio Amieiro,7,NULL,2018-06-04 09:13:58.670,Brazil,14496,258,678,http://flavioamieiro.com,612
1100,,NULL,2008-08-12 11:58:29.663,Daniel Silveira,14,NULL,2013-09-14 02:52:14.097,Brazil,15930,295,1461,http://enerjy.sys-con.com/author/6412,839
1213,,NULL,2008-08-13 13:18:30.890,André Casteliano,1,NULL,2018-08-29 17:37:35.787,Brazil,432,131,76,NULL,922
2015,"<p>Product Manager and Developer at a financial technology provider.</p> <p>Living and working in Rio de Janeiro, Brazil</p>",NULL,2008-08-19 19:42:43.420,Gustavo,12,NULL,2018-08-21 13:45:51.207,Brazil,602,576,235,,1509
2267,,NULL,2008-08-21 12:23:14.943,João Vieira,0,NULL,2018-09-01 13:26:59.693,Brazil,932,197,102,http://www.sia.net.br,1679
2274,NULL,NULL,2008-08-21 12:44:42.223,Fernando Barrocal,71,NULL,2018-08-31 14:46:27.387,Brazil,6285,107,529,http://fernando.barrocal.com/,1685
2333,,NULL,2008-08-21 16:56:11.823,Felipe,0,NULL,2008-09-28 07:38:25.930,Brazil,57,5,14,http://felipc.com,1725
3031,Developer!,NULL,2008-08-26 12:46:22.053,zaca,0,NULL,2012-10-09 13:20:45.547,Brazil,525,115,53,http://thebrainfarm.net,2204


Wait...something went wrong. There are users with a reputation over 1,000, but don't have "FAMOUS" in their AboutMe. A bunch of 'em simply got ignored.

# That's the bug.

When you declare variables and set them using one row from the DELETED virtual table, you have no idea which row you're going to get. Even worse, sometimes this trigger will update one row, and sometimes it won't - because it might happen to grab a row with a reputation under 1,000!

Here's a better way to write that trigger:



In [21]:
CREATE OR ALTER TRIGGER trUserIsFamous ON dbo.Users
AFTER UPDATE
AS
BEGIN
   UPDATE dbo.Users
   SET AboutMe = 'FAMOUS FOR A HIGH REPUTATION!'
   FROM dbo.Users u
   INNER JOIN INSERTED i ON u.Id = i.Id 
   WHERE i.Reputation > 1000;
END
GO

Commands completed successfully.

Total execution time: 00:00:00.0036070

In that version, we're simply joining with the INSERTED table, thereby ensuring that we get all of the updated rows. We operate on 'em all at once, too - think set-based, not a cursor.

Then, when we update multiple rows at once:

In [22]:
UPDATE dbo.Users
  SET Reputation = Reputation + 1
  WHERE Location = 'Brazil';

(380 rows affected)

(4250 rows affected)

Total execution time: 00:00:00.1190870

The right ones are affected:

In [23]:
SELECT TOP 20 *
  FROM dbo.Users
  WHERE Location = 'Brazil';

(20 rows affected)

Total execution time: 00:00:00.0071990

Id,AboutMe,Age,CreationDate,DisplayName,DownVotes,EmailHash,LastAccessDate,Location,Reputation,UpVotes,Views,WebsiteUrl,AccountId
626,FAMOUS FOR A HIGH REPUTATION!,NULL,2008-08-07 11:52:05.957,Douglas Tosi,3,NULL,2018-01-12 16:44:17.287,Brazil,1814,154,287,http://www.sinatica.com,484
727,FAMOUS FOR A HIGH REPUTATION!,NULL,2008-08-08 12:16:17.820,Fabio Gomes,45,NULL,2018-09-01 18:52:56.917,Brazil,3668,596,800,http://yacoding.blogspot.com,561
802,FAMOUS FOR A HIGH REPUTATION!,NULL,2008-08-09 02:50:52.577,Flávio Amieiro,7,NULL,2018-06-04 09:13:58.670,Brazil,14497,258,678,http://flavioamieiro.com,612
1100,FAMOUS FOR A HIGH REPUTATION!,NULL,2008-08-12 11:58:29.663,Daniel Silveira,14,NULL,2013-09-14 02:52:14.097,Brazil,15931,295,1461,http://enerjy.sys-con.com/author/6412,839
1213,,NULL,2008-08-13 13:18:30.890,André Casteliano,1,NULL,2018-08-29 17:37:35.787,Brazil,433,131,76,NULL,922
2015,"<p>Product Manager and Developer at a financial technology provider.</p> <p>Living and working in Rio de Janeiro, Brazil</p>",NULL,2008-08-19 19:42:43.420,Gustavo,12,NULL,2018-08-21 13:45:51.207,Brazil,603,576,235,,1509
2267,,NULL,2008-08-21 12:23:14.943,João Vieira,0,NULL,2018-09-01 13:26:59.693,Brazil,933,197,102,http://www.sia.net.br,1679
2274,FAMOUS FOR A HIGH REPUTATION!,NULL,2008-08-21 12:44:42.223,Fernando Barrocal,71,NULL,2018-08-31 14:46:27.387,Brazil,6286,107,529,http://fernando.barrocal.com/,1685
2333,,NULL,2008-08-21 16:56:11.823,Felipe,0,NULL,2008-09-28 07:38:25.930,Brazil,58,5,14,http://felipc.com,1725
3031,Developer!,NULL,2008-08-26 12:46:22.053,zaca,0,NULL,2012-10-09 13:20:45.547,Brazil,526,115,53,http://thebrainfarm.net,2204


tl;dr: when you see the variables being set with the contents of a single row from the DELETED and INSERTED tables, you need to raise an urgent alarm because that trigger probably produces incorrect results when multiple rows are deleted/updated/inserted in a single statement.